In [ ]:
# instalattion easyOCR modul
!pip install easyocr
# connect with Google Drive
from google.colab import drive
drive.mount( '/content/gdrive' )

In [ ]:
# instalattion opencv modul
!pip install opencv-python

In [47]:

import sys
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
# Detection Licence plate on image
class LPDetector:
    # init - constructor
    def __init__(self, tezine: str, konfig: str, klase: str):
        self.net = cv2.dnn.readNet(tezine, konfig)
        self.classes = []
        with open(klase, 'r') as f:
            self.classes = f.read().splitlines()
        self.font = cv2.FONT_HERSHEY_PLAIN
        self.color = (255, 0, 0)
        self.coordinates = None
        self.img = None
        self.fig_image = None
        self.roi_image = None
        
    # image loading and License plate detection method
    def detekcija(self, slike: str):
        # Image loading
        orig = cv2.imread(slike)
        self.img = orig
        img = orig.copy()
        # Image dimension
        height, width, _ = img.shape
        blob = cv2.dnn.blobFromImage(img, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
        self.net.setInput(blob)
        izlaz_sloj = self.net.getUnconnectedOutLayersNames()
        izlaz = self.net.forward(izlaz_sloj)
        okviri = []
        verovatnoca = []
        klasa = []
        # Object selection and extraction of coordinates and LP dimensions
        for izl in izlaz:
            for det in izl:
                skor = det[5:]
                klas = np.argmax(skor) 
                verov = skor[klas]
                if verov > 0.2:
                    centar_x = int(det[0] * width)
                    centar_y = int(det[1] * height)
                    w = int(det[2] * width)
                    h = int(det[3] * height)
                    x = int(centar_x - w / 2)
                    y = int(centar_y - h / 2)

                    okviri.append([x, y, w, h])
                    verovatnoca.append((float(verov)))
                    klasa.append(klas)

        indeksi = cv2.dnn.NMSBoxes(okviri, verovatnoca, 0.2, 0.4)

        if len(indeksi) > 0:
            for i in indeksi.flatten():
                x, y, w, h = okviri[i]
                label = str(self.classes[klasa[i]])
                verov = str(round(verovatnoca[i],2))
                # drawing a rectangle around the LP
                cv2.rectangle(img, (x,y), (x + w, y + h), self.color, 3)
                #print the estimated probability above the LP
                #cv2.putText(img, label + ' ' + confidence, (x, y + 10), self.font, 2, (255, 255, 255), 3)
        self.fig_image = img
        self.coordinates = (x, y, w, h)
        return
    
    # cropping the part of the image that is rectangled
    def iseci(self):
        x, y, w, h = self.coordinates
        roi = self.img[y:y + h, x:x + w]
        self.roi_image = roi
        return
# Forwarding classes of LPDetector weights, configuration and classes after network training
lpd = LPDetector(
    tezine="/content/gdrive/My Drive/yolo_licence_plate/yolov3-train_final.weights", 
    konfig="/content/gdrive/My Drive/yolo_licence_plate/yolov3-train.cfg", 
    klase="/content/gdrive/My Drive/yolo_licence_plate/classes.txt"
)
    
# New LP detection
lpd.detekcija("/content/0016.JPG")

# Original image with rectangled LP
plt.figure(figsize=(24, 24))
plt.imshow(cv2.cvtColor(lpd.fig_image, cv2.COLOR_BGR2RGB))
#plt.savefig('detected.jpg')
plt.show()

# Cropped LP and show image
lpd.iseci()
plt.figure(figsize=(6, 4))
plt.axis('off')
plt.imshow(cv2.cvtColor(lpd.roi_image, cv2.COLOR_BGR2RGB))
plt.savefig('croped.jpg')

Output hidden; open in https://colab.research.google.com to view.

In [26]:

import easyocr
import tensorflow as tf 
import matplotlib.pyplot as plt
import cv2 
import numpy as np
from pylab import rcParams
from IPython.display import Image
rcParams['figure.figsize']=8,16
import cv2
from tensorflow.keras import backend as K
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from tensorflow.keras.models import Model, load_model

In [ ]:
# language picking-modul easyocr
reader = easyocr.Reader(['en'])
# loading image
img = cv2.imread("croped.jpg")
# show image
from google.colab.patches import cv2_imshow
cv2_imshow(img)
# text recognition
output = reader.readtext(img)
# the coordinates of the upper left and lower right points of the text to eliminate unnecessary parts of the image
cord=output[-1][0]
x_min,y_min = [min(idx) for idx in zip(*cord)]
x_max,y_max = [max(idx) for idx in zip(*cord)]
# rectangle around image
box = cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(0,0,255),2)
cv2_imshow(box)
# cropped image with characters
roi = box[y_min:y_max, x_min:x_max]
cv2_imshow(roi)
# transformation color image into gray scale
imgray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
# binarization image
ret, thresh = cv2.threshold(imgray, 90, 255, 0)
# finding individual characters
ctrs, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# sorted list of characters
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
c = 0
# new folder for images of individual characters
!mkdir individual_characters
# go through all the rectangled characters, cropp and place in the individual_characters folder
for i, ctr in enumerate(sorted_ctrs):
    x, y, w, h = cv2.boundingRect(ctr)

    roii = roi[y:y + h, x:x + w]
    # elimination of contours that do not meet the dimensions of the character
    area = w*h
    if 500 < area < 4000:
        c =c+1
        rect = cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2_imshow(rect)
        num = rect[y:y + h, x:x + w]
        num = cv2.cvtColor(num, cv2.COLOR_BGR2GRAY)
        ret, num = cv2.threshold(num, 90, 255, 0)
        num = cv2.bitwise_not(num)
        res = cv2.resize(num,(60,90))
        cv2_imshow(res)
        cv2.imwrite("individual_characters/number"+str(c)+".jpg",res)
cv2.waitKey(0)

In [ ]:
# individual character recognition

# list of character
labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "Ć", "Č", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "R", "S", "Š", "T", "U", "V", "W", "X", "Y", "Z", "Ž", "Đ"]
# CNN model for character recognition
prediction_model=load_model("/content/gdrive/My Drive/final_model_char.h5")
# path to the folder with individual_characters
path = "/content/individual_characters"
# sorted list
filesu = os.listdir(path)
files = sorted(filesu)
#print(files)
predikcija=[]
# go through the all images, character clasification and showing text
for X in files:
  im = "/content/individual_characters/"+str(X)
  img = cv2.imread(im)
  img = np.expand_dims(img, axis=0)
  dimensions = img.shape
  img = tf.image.resize( img, [60,90] )
  Y = prediction_model.predict([img])
  #print(Y)
  preds_class_prob = np.argmax(Y, axis=1) 
  class_labels = labels[int(preds_class_prob)]
  #print(class_labels)
  predikcija.append(class_labels)
print(predikcija)
